In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import os
from neo4j import GraphDatabase

os.environ["GOOGLE_API_KEY"] = os.environ["GEMINI_API"]

# Initialize the LLM (ensure your API key is set in your environment)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    max_tokens=None,
    timeout=None
)

/Users/jho/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fair_use_summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a legal analyst. Read the following legal opinion and summarize its significance with respect to its Fair Use ruling. Focus on the key points, include a general overview of the facts of the case and the context, as well as the implications regarding Fair Use.\n\n""",
        ),
        ("human", "Here is the legal opinion: {legal_opinion}"),
    ]
)

fair_use_summary_chain = fair_use_summary_prompt | llm

fair_use_relation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a legal expert. Given the summary of a legal opinion regarding Fair Use and the description of an undecided case dispute, explain how the legal opinion relates to the dispute and what implications it might have on the case.\n\n  Evaluate the following legal opinion with respect to the four Fair Use factors. State whether the legal opinion provides support for Fair Use, is against Fair Use, or is not relevant for the Fair Use analysis."
        ),
        ("human", "Summary: {summary}\n\nDispute: {dispute}"),
    ]
)

fair_use_relation_chain = fair_use_relation_prompt | llm


In [3]:
def process_legal_opinion(legal_opinion: str, dispute: str):
    
    # 1. Summarize the legal opinion.
    summary = fair_use_summary_chain.invoke(
        {
            "legal_opinion": legal_opinion
        }
    ).content
    
    # 2. Evaluation of the summary and dispute.
    evaluation = fair_use_relation_chain.invoke( {
            "summary": summary,
            "dispute": dispute
        }
    ).content
    
    return {
        "summary": summary,
        "evaluation": evaluation
    }


In [5]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "fairusecases")

def get_opinions(tx):
    
    results = tx.run("""
        MATCH (o:Opinion)-[]-(c:Case)
        RETURN c.WestLawCaseName AS CaseName, o.Document AS Document
        """
    )
         
    return results.to_df()


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session(database="neo4j") as session:
            opinions = session.execute_read(get_opinions)

In [10]:
dispute = """
A YouTube channel called NerdVerse creates a whimsical animated short that merges the worlds of Dr. Seuss and Harry Potter. The video, titled Oh, the Spells You’ll Cast!, presents a rhyming, Seussian retelling of Harry Potter’s adventures with visuals closely resembling the distinctive Seuss style.

The animation quickly goes viral, attracting millions of views and sparking discussions about nostalgia and fan creativity. The creators of NerdVerse argue that the mashup is a love letter to both franchises and was made purely for fun.

However, Seuss Studios, which owns the rights to Dr. Seuss’s works, files a copyright claim, stating that the video borrows too much from their original artistic style and writing without permission.
"""

In [11]:
document = opinions["Document"][0]

In [12]:
output = process_legal_opinion(document, dispute)

In [13]:
output

{'summary': 'This legal opinion, *Peter Letterese and Associates, Inc. v. World Institute of Scientology Enterprises, International*, centers on a copyright infringement dispute concerning the book *Big League Sales Closing Techniques* by Les Dane. Peter Letterese & Associates (PL&A), the copyright licensee, sued three Church of Scientology-affiliated entities, arguing that their instructional course materials infringed on the book\'s copyright.\n\n**Facts and Context:**\n\n*   Les Dane\'s book was used by the Church of Scientology to train registrars (those who sell Scientology books and services).\n*   The Church developed course materials, including "checksheets" and "drill sheets," that incorporated elements of the book.\n*   PL&A acquired the exclusive rights to Dane\'s works and sued the Church-affiliated entities for copyright infringement.\n*   The district court initially ruled in favor of the defendants, citing fair use and laches (unreasonable delay in pursuing a legal claim